In [1]:
/**
 * Demonstrate how to train an artificial neural network (ANN) to learn the Black-Scholes formula.
 * We first create a training set using the actual Black-Scholes formula.
 * Then we feed the training set to an ANN so that it can try to learn the BS formula from the data.
 * The result is very good. The trained ANN model produces very similar results as the BS formula.
 *
 * @author Haksun Li
 */

%use s2, deeplearning4j, lets-plot

import org.nd4j.linalg.lossfunctions.LossFunctions

val rng = UniformRNG()
rng.seed(1234567890L)

/**
 * Compute the call option price according to the Black-Scholes formula.
 * This is the target function that we want the ANN to learn.
 *
 * @param s spot price of the underlying asset
 * @param x strike price of the option
 * @param r risk free rate
 * @param sigma the volatility of returns of the underlying asset
 * @param tau time to maturity
 */
fun BlackScholesCall(s: Double, x: Double, r: Double, sigma: Double, tau: Double): Double {
    val normal = NormalDistribution()
    val d1 = (ln(s / x) + (r + sigma * sigma / 2.0) * tau) / (sigma * sqrt(tau))
    val d2 = d1 - sigma * sqrt(tau)
    val c= s * normal.cdf(d1) - x * exp(- r * tau) * normal.cdf(d2)
    return c
}

/**
 * Generate a random set of data of option prices for a range of parameters.
 * This is the training set fed to the ANN.
 */
fun generateData(N: Int,
                 sRange: DoubleArray,
                 xRange: DoubleArray,
                 rRange: DoubleArray,
                 sigmaRange: DoubleArray,
                 tauRange: DoubleArray): DataSet {
    val ds = sRange[1] - sRange[0]
    val dx = xRange[1] - xRange[0]
    val dr = rRange[1] - rRange[0]
    val dsigma = sigmaRange[1] - sigmaRange[0]
    val dtau = tauRange[1] - tauRange[0]
    
    val xs = Nd4j.zeros(N, 5)
    val ys = Nd4j.zeros(N, 1)
    for (i in 0 until N) {
        // the random factors
        val s = rng.nextDouble() *  ds
        val x = rng.nextDouble() * dx 
        val r = rng.nextDouble() * dr
        val sigma = rng.nextDouble() * dsigma
        val tau = rng.nextDouble() * dtau

        val callPrice = BlackScholesCall(s, x, r, sigma, tau)
        // the independent variables
        xs.putScalar(intArrayOf(i, 0), s)
        xs.putScalar(intArrayOf(i, 1), x)
        xs.putScalar(intArrayOf(i, 2), r)
        xs.putScalar(intArrayOf(i, 3), sigma)
        xs.putScalar(intArrayOf(i, 4), tau)
        
        // the dependent variable
        ys.putScalar(intArrayOf(i, 0), callPrice)
    }

    return DataSet(xs, ys)
}

/*
 * generate the training set
 */
val trainSize = 2000 // number of data points
val trainSet = generateData(trainSize,
                            doubleArrayOf(20.0, 30.0),
                            doubleArrayOf(12.0, 17.0),
                            doubleArrayOf(0.01, 0.03),
                            doubleArrayOf(0.3, 0.4),
                            doubleArrayOf(0.25, 0.5))

/*
 * build and configure an ANN
 */
// number of epochs (full passes of the data)
val nEpochs = 1500
// batch size: i.e., each epoch has nSamples/batchSize parameter updates
val batchSize = 200
// network learning rate
val learningRate = 0.0001
val conf = NeuralNetConfiguration.Builder()
    .seed(0)
    .weightInit(WeightInit.XAVIER)
    .updater(Adam(learningRate))
    .list()
    .layer(0, DenseLayer.Builder().nIn(5).nOut(10).weightInit(WeightInit.XAVIER).activation(Activation.RELU).build()) // first hidden layer
    .layer(1, OutputLayer.Builder().nIn(10).nOut(1).weightInit(WeightInit.XAVIER).activation(Activation.IDENTITY) // output layer
               .lossFunction(LossFunctions.LossFunction.MSE)
               .build())
    .build()
val net = MultiLayerNetwork(conf)
net.init()
val iterator = ListDataSetIterator<DataSet>(trainSet.asList(), batchSize)

/*
 * train an ANN
 */
val scores = ArrayList<Double>(nEpochs)
var lastScore = 0.0
var sameForNEpochs = 0
for (i in 0 until nEpochs) {
    iterator.reset()
    net.fit(iterator)
    val score = net.score()
    scores.add(score)
    println("epoch: $i, score: $score")
    if (abs(lastScore - score) < 1e-6) {
        if (++sameForNEpochs > 4) {
            println("Score hasn't changed for 5 epochs. Early stop.")
            break
        }
    }
    sameForNEpochs = 0
    lastScore = score
}

/*
 * test the ANN and print out results
 */
val s1 = 23.75 // the strike
val x1 = 15.00 // the spot
val r1 = 0.01 // the interest rate
val sigma1 = 0.35 // the volatility
val tau1 = 0.5 // the time to maturity
println("Given stock price:                  $s1")
println("      strike price:                 $x1")
println("      risk-free rate:               $r1")
println("      volatility:                   $sigma1")
println("      time to maturity:             $tau1")
println("=============================================")
println("option price by the Black-Scholes model: ${BlackScholesCall(s1, x1, r1, sigma1, tau1)}")
val input = Nd4j.create(doubleArrayOf(s1, x1, r1, sigma1, tau1), 1, 5)
println("option price by the ANN model:           ${net.output(input, false).getDouble(0)}")

// plot the convergence curve
val plotData = mapOf("Epoch" to (1..(scores.size)).toList(),
                     "MSE" to scores)
val p = lets_plot(plotData) + ggsize(500, 250)
p + geom_line() { x = "Epoch"; y = "MSE" }

17:21:32.904 [main] INFO  org.nd4j.linalg.factory.Nd4jBackend,203 - Loaded [CpuBackend] backend
17:21:33.376 [main] INFO  org.nd4j.nativeblas.NativeOpsHolder,109 - Number of threads used for linear algebra: 4
17:21:33.378 [main] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory,108 - *********************************** CPU Feature Check Warning ***********************************
17:21:33.379 [main] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory,109 - Warning: Initializing ND4J with Generic x86 binary on a CPU with AVX512 support
17:21:33.380 [main] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory,110 - Using ND4J with AVX512 will improve performance. See deeplearning4j.org/cpu for more details
17:21:33.381 [main] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory,111 - Or set environment variable ND4J_IGNORE_AVX=true to suppress this warning
17:21:33.382 [main] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory,112 - *************************************************************************************************
17:21:33